In [1]:
# import lib
from __future__ import print_function
import numpy as np 
import math
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow import keras
import time

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# data processing
%run 'Data Augmentation.ipynb'
print ("number of training examples = " + str(X_train.shape[0]))
#print ("number of validation examples = " + str(X_val.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_val shape: " + str(X_val.shape))
print ("Y_val shape: " + str(Y_val.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

Loading training set...
Done
Loading test set...
Done
Splitting into val/test set...
Done
Normalization...
Done
Converting to one-hot code...
Done
Generating Gaussian noise images...
Done
Stacking Gaussian noise images...
Done
Generating perspective images...
Done
Stacking perspective images...
Done
Finished!
number of training examples = 86208
number of test examples = 4832
X_train shape: (86208, 100, 100, 3)
Y_train shape: (86208, 60)
X_val shape: (4839, 100, 100, 3)
Y_val shape: (4839, 60)
X_test shape: (4832, 100, 100, 3)
Y_test shape: (4832, 60)


In [3]:
# model building
def build_model():
    base_model = keras.applications.VGG16(input_shape=(100,100,3), include_top=False, weights='imagenet')
    base_model_output = base_model.output
    x = keras.layers.Flatten(name='flatten')(base_model_output)
    #x = keras.layers.Dense(1024, activation='relu', name='fc1')(x)
    #x = keras.layers.Dropout(0.25, name='dropout1')(x)
    #x = keras.layers.Dense(256, activation='relu', name='fc2')(x)
    #x = keras.layers.Dropout(0.5, name='dropout2')(x)
    out = keras.layers.Dense(60, name='output', activation='softmax')(x)
    
    for layer in base_model.layers:
        layer.trainable = False

    model = keras.models.Model(inputs=base_model.input, outputs=out)
    return model

In [4]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0         
__________

In [5]:
model.compile(
    loss='categorical_crossentropy', 
    metrics=['accuracy'], 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001,beta1=0.9,beta2=0.999,epsilon=1e-08,use_locking=False))

In [ ]:
start=time.time()

history = model.fit(
    X_train,
    Y_train, 
    batch_size = 512, 
    epochs = 32, 
    verbose = 1, 
    shuffle = True,
    validation_data=(X_val, Y_val))

end=time.time()
runtime=end-start
print("Runtime: "+str(runtime)+"s")

Train on 86208 samples, validate on 4839 samples
Epoch 1/32
86208/86208 [==============================] - 75s - loss: 1.2640 - acc: 0.7911 - val_loss: 0.4503 - val_acc: 0.9206
Epoch 2/32
86208/86208 [==============================] - 74s - loss: 0.3328 - acc: 0.9650 - val_loss: 0.2698 - val_acc: 0.9475
Epoch 3/32
86208/86208 [==============================] - 72s - loss: 0.1868 - acc: 0.9828 - val_loss: 0.2160 - val_acc: 0.9531
Epoch 4/32
86208/86208 [==============================] - 72s - loss: 0.1251 - acc: 0.9901 - val_loss: 0.1710 - val_acc: 0.9624
Epoch 5/32
86208/86208 [==============================] - 72s - loss: 0.0918 - acc: 0.9934 - val_loss: 0.1575 - val_acc: 0.9628
Epoch 6/32
86208/86208 [==============================] - 72s - loss: 0.0706 - acc: 0.9958 - val_loss: 0.1412 - val_acc: 0.9642
Epoch 7/32
86208/86208 [==============================] - 72s - loss: 0.0562 - acc: 0.9969 - val_loss: 0.1359 - val_acc: 0.9645
Epoch 8/32
86208/86208 [==============================]

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'],loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
start=time.time()

scores = model.evaluate(
    test_images,
    test_labels, 
    batch_size = 32, 
    verbose = 1, 
    sample_weight=None, 
    steps=None)

print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

end=time.time()
runtime=end-start
print("Runtime: "+str(runtime)+"s")